## Import Libraries

In [1]:
from quspin.operators import hamiltonian
from quspin.basis import spin_basis_1d
from quspin.basis import spinless_fermion_basis_1d
import numpy as np
import matplotlib.pyplot as plt
import itertools
plt.rcParams['figure.figsize'] = [10, 8]

## Define Constants/Helpers

In [2]:
N_group = 2
N = 4
N_filled = N // 2
P_neg1 = 1
t = -0.5
g = 1
m = 1
PBC = True

In [3]:
def stringbox(lst, caption):
    return caption + "-" * (100 - len(caption))+"\n" + str(lst) + "\n" + "-" * 100

## Spin Basis Simulation

In [8]:
spin_basis = spin_basis_1d(N, pauli=-1, Nup=N_filled)
static_terms = []
dynamic_terms = []

### Hopping term
L = (N if PBC else N-1)
hop_coupling = [[-t, i, (i+1) % N] for i in range(L)]
hop_coupling_dag = [[-t, (i+1) % N, i] for i in range(L)]
if PBC and N_filled % 2 == 0:
    hop_coupling[-1][0] = -hop_coupling[-1][0]
    hop_coupling_dag[-1][0] = -hop_coupling_dag[-1][0]
static_terms += [["+-", hop_coupling]]
static_terms += [["+-", hop_coupling_dag]]

### Mass term
mass_coupling = [[m * (-1) ** i, i, i] for i in range(N)]
static_terms += [["+-", mass_coupling]]

### E-field term
def powerset(seq):
    if len(seq) == 1:
        yield seq
    else:
        for item in powerset(seq[1:]):
            yield [seq[0]]+item
            yield item
def P(n, dag=False):
    i_parity = (-1 if dag else 1)
    phase = np.exp(1j * (1 - n % 2) * np.pi / N_group * i_parity)
    P_coupling = []
    for seq in powerset(list(range(n + 1))):
        seq = list(seq)
        coupling = np.cos(np.pi / N_group)**(n - len(seq) + 1) * (np.sin(np.pi / N_group) * i_parity * 1j)**len(seq)
        coupling *= P_neg1 * -g
        couple_list = [coupling] + seq
        P_coupling += [["z" * len(seq)] + [[couple_list]]]
    return P_coupling
    
for i in range(N):
    static_terms += P(i, dag=False)
    static_terms += P(i, dag=True)

print(stringbox(static_terms, "Static couplings:"))
### Plotting spectrum
H = hamiltonian(static_terms, dynamic_terms, basis=spin_basis, dtype=np.float64)
spin_eigvals = H.eigvalsh()
plt.plot(spin_eigvals, ".")
plt.show()

[['z', [[(-0-1j), 0]]]]
[['z', [[1j, 0]]]]
[['zz', [[(1-0j), 0, 1]]], ['z', [[(-0-6.123233995736766e-17j), 1]]]]
[['zz', [[(1-0j), 0, 1]]], ['z', [[6.123233995736766e-17j, 1]]]]
[['zzz', [[1j, 0, 1, 2]]], ['zz', [[(6.123233995736766e-17-0j), 1, 2]]], ['zz', [[(6.123233995736766e-17-0j), 0, 2]]], ['z', [[(-0-3.749399456654644e-33j), 2]]]]
[['zzz', [[(-0-1j), 0, 1, 2]]], ['zz', [[(6.123233995736766e-17-0j), 1, 2]]], ['zz', [[(6.123233995736766e-17-0j), 0, 2]]], ['z', [[3.749399456654644e-33j, 2]]]]
[['zzzz', [[(-1+0j), 0, 1, 2, 3]]], ['zzz', [[6.123233995736766e-17j, 1, 2, 3]]], ['zzz', [[6.123233995736766e-17j, 0, 2, 3]]], ['zz', [[(3.749399456654644e-33-0j), 2, 3]]], ['zzz', [[6.123233995736766e-17j, 0, 1, 3]]], ['zz', [[(3.749399456654644e-33-0j), 1, 3]]], ['zz', [[(3.749399456654644e-33-0j), 0, 3]]], ['z', [[(-0-2.2958450216584675e-49j), 3]]]]
[['zzzz', [[(-1+0j), 0, 1, 2, 3]]], ['zzz', [[(-0-6.123233995736766e-17j), 1, 2, 3]]], ['zzz', [[(-0-6.123233995736766e-17j), 0, 2, 3]]], ['zz

TypeError: expecting string type for opstr

## Fermion Basis

In [ ]:
fermion_basis = spinless_fermion_basis_1d(L=N, Nf=N_filled)
static_terms = []
dynamic_terms = []

### Hopping term
L = (N if PBC else N-1)
hop_coupling = [[-t, i, (i+1) % N] for i in range(L)]
hop_coupling_dag = [[-t, (i+1) % N, i] for i in range(L)]
static_terms += [["+-", hop_coupling]]
static_terms += [["+-", hop_coupling_dag]]

### Mass term
mass_coupling = [[m * (-1) ** i, i, i] for i in range(N)]
static_terms += [["+-", mass_coupling]]

### E-field term
E_link_parity = lambda x : -1 if (x % 4) < 2 else 1
for i in range(N):
    E_coupling = [P_neg1 * -g * E_link_parity(i) * 2 ** (i+1)] + list(range(i+1))
    static_terms += [["z" * (i+1), [E_coupling]]]

print(stringbox(static_terms, "Static couplings:"))
### Plotting spectrum
H = hamiltonian(static_terms, dynamic_terms, basis=fermion_basis, dtype=np.float64)
fermion_eigvals = H.eigvalsh()
plt.plot(fermion_eigvals, ".")
plt.show()

## Comparison

In [ ]:
mdiff = max(fermion_eigvals - spin_eigvals)
print(stringbox(fermion_eigvals, "Fermion eigenvalues:"))
print(stringbox(spin_eigvals, "Spin eigenvalues:"))
print("Maximum difference between eigenvalues:", mdiff)